In [1]:
%matplotlib notebook

import cv2, matplotlib
import numpy as np

from skimage.morphology import skeletonize, thin
from skimage.util import img_as_ubyte

import matplotlib.pyplot as plt

from os.path import expanduser, splitext
from os import scandir, makedirs

import random

import csv

from tqdm import tnrange, tqdm_notebook

In [2]:
def read_from_csv(filepath):
    readlist = []

    with open(filepath, 'r', newline='', encoding='utf-8-sig') as csvfile:
        listreader = csv.reader(csvfile)
        rows = list(listreader)

    columns = rows[0]

    for item in rows[1:]:
        readlist.append(tuple(item))

    return columns, readlist

def read_bgr_from_image_unicode(path):
    '''workaround for non-ascii filenames'''
    
    stream = open(path, "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    bgr = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
    
    return bgr

def save_bgr_to_image_unicode(bgr, path, ext_to='.png'):
    '''workaround for non-ascii filenames'''

    _, numpyarray = cv2.imencode(ext_to, bgr)
    with open(path, "wb") as file:
        file.write(numpyarray)

# unit mask

In [ ]:
def color_dict_mask(img_dict={'Lab': np.zeros((1,1,3),dtype='uint8'),
                              'HSV': np.zeros((1,1,3),dtype='uint8')},
                    colors={'colorname': {'Lab': ([0,0,0], [255,255,255]),
                                          'HSV': ([0,0,0], [255,255,255])}}):
    # get masks matching any of the colors matching all descriptions

    mask = np.zeros_like(list(img_dict.values())[0][:,:,0])
    for color_dict in colors.values():
        mask_color = np.ones_like(mask)*255
        for colorspace, limits in color_dict.items():
            mask_colorspace = cv2.inRange(img_dict[colorspace], \
                                          np.array(limits[0]), np.array(limits[1]))
            mask_color = cv2.bitwise_and(mask_color, mask_colorspace)

        mask = cv2.bitwise_or(mask, mask_color)

    return mask


def get_color_mask(bgr=np.zeros((1,1,3),dtype='uint8'), \
                   colors={'colorname': {'Lab': ([0,0,0], [255,255,255]),
                                         'HSV': ([0,0,0], [255,255,255])}}):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2Lab)

    blur = {}
    blur['Lab'] = cv2.bilateralFilter(lab,15,25,150)
    blur['BGR'] = cv2.cvtColor(blur['Lab'], cv2.COLOR_Lab2BGR)
    blur['HSV'] = cv2.cvtColor(blur['BGR'], cv2.COLOR_BGR2HSV)

    # get masks matching any of the colors matching all descriptions

    mask = color_dict_mask(blur, colors)

    # fill holes and remove noise


    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, \
                                              cv2.CHAIN_APPROX_NONE)

    holes = [contours[i] for i in range(len(contours)) if hierarchy[0][i][3]>=0]
    cv2.drawContours(mask, holes, -1, 255, -1)

    kernel_5c = np.array([
        [0,1,1,1,0],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [0,1,1,1,0]
        ], dtype=np.uint8)

    kernel_9c = np.zeros((9,9), np.uint8)
    cv2.circle(kernel_9c, (4,4), 4, 1, -1)

    kernel_15c = np.zeros((15,15), np.uint8)
    cv2.circle(kernel_15c, (7,7), 7, 1, -1)

    #mask = cv2.erode(mask, kernel_5c, iterations=1)

    smallbits = [contours[i] for i in range(len(contours)) \
                 if hierarchy[0][i][3]==-1 and cv2.contourArea(contours[i]) <= 100]
    cv2.drawContours(mask, smallbits, -1, 0, -1)

    # removing imperfections

    _, contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        if cv2.contourArea(c) >= 100:
            mask_single_c = np.zeros_like(mask)
            cv2.drawContours(mask_single_c, c, -1, 255, -1)

            mask_single_c = cv2.morphologyEx(mask_single_c, cv2.MORPH_CLOSE, kernel_9c, iterations=1)
            mask |= mask_single_c

    return mask


def get_marked_contours(contours, marker_mask, min_marked_area):
    marked_contours = []

    for c in contours:
        mask_single_c = np.zeros_like(marker_mask)
        cv2.drawContours(mask_single_c, [c], -1, 255, -1)

        c_area = cv2.countNonZero(mask_single_c)
        marked_area = cv2.countNonZero(mask_single_c & marker_mask)    

        if marked_area>=min_marked_area:
            marked_contours.append(c)

    return marked_contours


def get_marked_mask(boundary_mask, marker_mask, min_marked_area):
    _, contours, hierarchy = cv2.findContours( \
            boundary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    marked_contours = get_marked_contours(contours, marker_mask, min_marked_area)

    marked_mask = np.zeros_like(boundary_mask)

    if marked_contours:
        cv2.drawContours(marked_mask, marked_contours, -1, 255, -1)

    return marked_mask

def get_wall_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    
    # get mask based on color and shape
    
    redimg = bgr[:,:,2]
    _, threshold_img_inv = cv2.threshold(redimg, 140, 255, cv2.THRESH_BINARY_INV)
    
    kernel = np.ones((5,5), np.uint8)
    for i in [0,4]:
        for j in [0,4]:
            kernel[i][j] = 0
           
    erosion = cv2.erode(threshold_img_inv, kernel, iterations = 1)
    opening = cv2.morphologyEx(threshold_img_inv, cv2.MORPH_OPEN, kernel)
                
    envelope = np.zeros_like(threshold_img_inv)
    
    lines = cv2.HoughLinesP(erosion,1,np.pi/180,threshold=8,minLineLength=5,maxLineGap=0)

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(envelope,(x1,y1),(x2,y2),(0,255,0),25)
    
    mask = cv2.bitwise_or(opening, envelope)
    
        
    # get boundaries
    
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # get boundaries above minimum area

    wall_mask = np.zeros_like(mask)

    for i in range(len(contours)):
        holes = [contours[j] for j in range(len(contours)) if hierarchy[0][j][3] == i]

        cnt_img = np.zeros_like(mask)
        cv2.drawContours(cnt_img, contours, i, 255, -1) # linewidth of -1 fills
        cv2.drawContours(cnt_img, holes, -1, 0, -1) # index of -1 draws all

        #should be nothing        
        cnt_foreground = cv2.bitwise_and(cnt_img, mask)
        cnt_erosion2 = cv2.erode(cnt_foreground, kernel, iterations = 2)
                
        cnt_area = cv2.countNonZero(cnt_foreground)
        nothing_area = cv2.countNonZero(cnt_erosion2)
                
        if cnt_area >= 100 and (nothing_area/cnt_area) <= 0.5:
            wall_mask = cv2.bitwise_or(wall_mask, cnt_foreground)
    
    return wall_mask



def get_LDK_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    floor_colors = {'floor_light': {'Lab': ([180,130,160], [220,150,190]),
                                    'HSV': ([0,65,180], [20,255,255])}, 
                    'floor_dark': {'Lab': ([120,130,150], [180,155,190]),
                                   'HSV': ([0,90,100], [20,255,230])},
                    'floor_watermark': {'Lab': ([220,125,145], [240,145,165]),
                                        'HSV': ([0,65,220], [20,255,255])}}

    mask = get_color_mask(bgr, floor_colors)
    
    return mask

def get_bedroom_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    bedroom_boundary = {'bedroom_boundary': {'Lab': ([180,120,132], [254,135,165]),
                                             'HSV': ([10,25,200], [30,110,255])}}
    bedroom_dark = {'bedroom_dark': {'Lab': ([160,124,139], [250,130,165]),
                                     'HSV': ([10,30,200], [30,90,250])}}
    balcony_colors = {'balcony': {'Lab': ([240,125,130], [254,135,140])}}
    
    bedroom_boundary_mask = get_color_mask(bgr, bedroom_boundary)
    bedroom_dark_mask = get_color_mask(bgr, bedroom_dark)
    balcony_mask = get_color_mask(bgr, balcony_colors)

    # remove balconies which is similarily colored
    
    mask_bedroom_only = np.zeros_like(bedroom_boundary_mask)
    
    _, contours, _ = cv2.findContours(bedroom_boundary_mask, \
                                      cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for c in contours:
        mask_single_c = np.zeros_like(mask_bedroom_only)
        cv2.drawContours(mask_single_c, [c], -1, 255, -1)
        
        c_area = cv2.countNonZero(mask_single_c)
        dark_area = cv2.countNonZero(mask_single_c & bedroom_dark_mask)
        balcony_area = cv2.countNonZero(mask_single_c & balcony_mask)    
        
        if dark_area >= 1000:
                mask_bedroom_only |= mask_single_c
    return mask_bedroom_only

def get_balcony_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    balcony_boundary = {'bedroom_boundary': {'Lab': ([180,120,132], [254,135,165]),
                                             'HSV': ([10,15,200], [30,110,255])}}
    bedroom_dark = {'bedroom_dark': {'Lab': ([160,124,139], [250,130,165]),
                                     'HSV': ([10,30,200], [30,90,250])}}
    balcony_colors = {'balcony': {'Lab': ([240,125,130], [254,135,140])}}
    
    balcony_boundary_mask = get_color_mask(bgr, balcony_boundary)
    bedroom_dark_mask = get_color_mask(bgr, bedroom_dark)
    balcony_mask = get_color_mask(bgr, balcony_colors)

    # remain balconies only
    
    mask_balcony_only = np.zeros_like(balcony_boundary_mask)
    
    _, contours, _ = cv2.findContours(balcony_boundary_mask, \
                                      cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for c in contours:
        mask_single_c = np.zeros_like(mask_balcony_only)
        cv2.drawContours(mask_single_c, [c], -1, 255, -1)
        
        c_area = cv2.countNonZero(mask_single_c)
        dark_area = cv2.countNonZero(mask_single_c & bedroom_dark_mask)
        balcony_area = cv2.countNonZero(mask_single_c & balcony_mask)    
        
        if dark_area <= 10 <= balcony_area:
                mask_balcony_only |= mask_single_c
    return mask_balcony_only

def get_entrance_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    entrance_boundary = {'white_and_gray': {'HSV': ([0,0,170], [255,20,255])}}
    white = {'white': {'HSV': ([0,0,245], [255,10,255])}}
    gray = {'gray': {'HSV': ([0,0,230], [255,10,245])}}

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2Lab)

    blur = {}
    blur['Lab'] = cv2.bilateralFilter(lab,15,5,150)
    blur['BGR'] = cv2.cvtColor(blur['Lab'], cv2.COLOR_Lab2BGR)
    blur['HSV'] = cv2.cvtColor(blur['BGR'], cv2.COLOR_BGR2HSV)
    
    mask_e, mask_w, mask_g = [color_dict_mask(blur, x) \
                              for x in [entrance_boundary, white, gray]]

    area_e, area_w, area_g = [cv2.countNonZero(x) for x in [mask_e, mask_w, mask_g]]

    kernel3 = np.ones((3,3), np.uint8)
    mask_e_e = cv2.erode(mask_e, kernel3, iterations=1)

    _, contours, hierarchy = cv2.findContours(mask_e_e, cv2.RETR_CCOMP, \
                                              cv2.CHAIN_APPROX_NONE)
    
    kernel_15c = np.zeros((15,15), np.uint8)
    cv2.circle(kernel_15c, (7,7), 7, 1, -1)
    mask_w_d = cv2.dilate(mask_w, kernel_15c)
    mask_g_d = cv2.dilate(mask_g, kernel_15c)
    
    mask_ent = np.zeros_like(bgr[:,:,0])
    
    for i in range(len(contours)):
        if hierarchy[0][i][3] == -1:
            cnt = contours[i]
            mask_c = np.zeros_like(mask_ent)
            cv2.drawContours(mask_c, [cnt], -1, 255, -1)

            area_c = cv2.countNonZero(mask_c & mask_e)
            area_c_w = cv2.countNonZero(mask_c & mask_e & mask_w)
            area_c_w_d = cv2.countNonZero(mask_c & mask_e & mask_w_d)
            area_c_g = cv2.countNonZero(mask_c & mask_e & mask_g)
            area_c_g_d = cv2.countNonZero(mask_c & mask_e & mask_g_d)

            if area_c >= 0.01*area_g and \
                    area_c_w >= 0.3*area_c and area_c_g >= 0.3*area_c and \
                    area_c_w_d >= 0.8*area_c and area_c_g_d >= 0.8*area_c:
                mask_ent |= mask_c
    
    return mask_ent


def get_bathroom_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    bathroom_colors = {'bathroom': {'HSV': ([90,10,220], [110,40,255])}}

    mask = get_color_mask(bgr, bathroom_colors)
    
    return mask


def get_watershed(thresh=np.zeros((1,1),dtype='uint8'),
                  markers=np.zeros((1,1),dtype='uint8')):
    unknown = cv2.subtract(thresh, markers.astype(thresh.dtype))

    markers = markers.astype(np.int32)
    markers = markers+1
    markers[unknown==255] = 0

    markers = cv2.watershed(np.stack([thresh]*3, axis=2), markers)
    markers = markers-1
    markers[markers<=0] = 0

    return markers





def get_unit_mask(bgr=np.zeros((1,1,3),dtype='uint8')):
    """Returns unit plan masks of the unit plan, 
    as a dictionary of opencv masks and also a single combined mask,
    including masks for walls, entrances, LDK, bedrooms, balconies, and bathrooms."""

    AREA_UNIT = 128
    AREA_WALL = 64
    AREA_ENTRANCE = 32
    AREA_LDK = 16
    AREA_BEDROOM = 8
    AREA_BALCONY = 4
    AREA_BATHROOM = 2
    
    kernel_3 = np.ones((3,3),np.uint8)
    kernel_5c = np.array([
        [0,1,1,1,0],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [1,1,1,1,1],
        [0,1,1,1,0]
        ], dtype=np.uint8)
    kernel_7c = np.zeros((7,7), np.uint8)
    cv2.circle(kernel_7c, (3,3), 3, 1, -1)
    kernel_9c = np.zeros((9,9), np.uint8)
    cv2.circle(kernel_9c, (4,4), 4, 1, -1)
    kernel_15c = np.zeros((15,15), np.uint8)
    cv2.circle(kernel_15c, (7,7), 7, 1, -1)
    
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2Lab)
    blur = {}
    blur['Lab'] = cv2.bilateralFilter(lab,15,25,150)
    blur['BGR'] = cv2.cvtColor(blur['Lab'], cv2.COLOR_Lab2BGR)
    blur['HSV'] = cv2.cvtColor(blur['BGR'], cv2.COLOR_BGR2HSV)

    
    ######################################
    # Getting boundary of the unit floor #
    ######################################

    unit_boundary = {'not_gray': {'HSV': ([0,10,120], [255,255,254])},
                     'light_gray': {'HSV': ([0,0,210], [255,10,240])}}

    wall_colors = {'less_red': {'BGR': ([0,0,0], [200,200,160])}}

    unit_marker = get_color_mask(bgr, unit_boundary)
    # plt.imshow(unit_marker | bgr[:,:,2]&63)

    boundary_mask = get_color_mask(bgr, wall_colors)
    unit_marked_mask = get_marked_mask(boundary_mask, unit_marker, 100)
    unit_marked_mask_e = cv2.erode(unit_marked_mask, kernel_5c)

    boundary_mask = get_color_mask(bgr|~unit_marked_mask_e[:,:,None], wall_colors)
    unit_marked_mask_e = get_marked_mask(boundary_mask, unit_marker, 100)
    unit_marked_mask = cv2.dilate(unit_marked_mask_e, kernel_5c)
    # still including core & etc.

    unit_boundary_line_mask = unit_marked_mask & ~unit_marked_mask_e
    # plt.imshow(unit_marked_mask_e | bgr[:,:,2]&127)
    
        
    #####################################
    # Getting color based masks         #
    #####################################

    wall_mask = get_wall_mask(bgr)
    wall_and_boundary_mask = wall_mask | unit_boundary_line_mask
    floor_indoor_mask = unit_marked_mask_e & ~wall_mask
    # plt.imshow(wall_and_boundary_mask | bgr[:,:,2]&127)
    # plt.imshow(floor_indoor_mask | bgr[:,:,2]&127)

    ent_mask = get_entrance_mask(bgr)
    ent_mask_d = cv2.dilate(ent_mask, kernel_9c, iterations=2)
    # plt.imshow(ent_mask | bgr[:,:,2]&127)

    ldk_mask = get_LDK_mask(bgr)
    bed_mask = get_bedroom_mask(bgr)
    bal_mask = get_balcony_mask(bgr)

    bath_mask = get_bathroom_mask(bgr)

    ### expand bathroom marker and update unit marker

    bath_mask_d = cv2.dilate(bath_mask, kernel_9c)
    # plt.imshow(bath_mask | bgr[:,:,2]&127)
    bath_mask = get_watershed(
            unit_marked_mask_e & ~wall_mask & ~(unit_marker & ~bath_mask_d),
            bath_mask
        ).astype(np.uint8)
    # plt.imshow(wall_mask&2 | bath_mask&1)

    unit_marker |= cv2.erode(bath_mask, kernel_5c)

    
    #####################################
    # Combine all markers in uint8      #
    #####################################

    area_pairs = [# (wall_mask, AREA_WALL),
                  # (ent_mask, AREA_ENTRANCE), 
                  (ldk_mask, AREA_LDK), 
                  (bed_mask, AREA_BEDROOM), 
                  (bal_mask, AREA_BALCONY), 
                  (bath_mask, AREA_BATHROOM)]

    ret, markers = cv2.connectedComponents(unit_marker)
    # plt.imshow(markers)

    temp_markers = np.zeros_like(unit_marker)

    for i in range(1, ret):
        for area_mask, area_bit in area_pairs:
            if (markers==i).sum():
                if ((markers==i)&area_mask).sum()/(markers==i).sum() > 0.5:
                    temp_markers |= img_as_ubyte(markers==i)&area_bit

    area_markers = temp_markers
    area_markers |= ent_mask&AREA_ENTRANCE

    # plt.imshow(area_markers)

    #####################################
    # Getting skeleton of the unit plan #
    #####################################

    skeleton = skeletonize(~wall_and_boundary_mask & 1)

    skeleton = img_as_ubyte(skeleton)
    skeleton = cv2.dilate(skeleton, kernel_3)

    skeleton_split = skeleton & ~(ent_mask_d&~ent_mask)

    # plt.imshow(skeleton | bgr[:,:,2]&127)

    ret, markers = cv2.connectedComponents(skeleton)

    # plt.imshow(markers)

    temp_skeleton = np.zeros_like(markers)

    for i in range(1, ret):
        if ((markers==i)&ent_mask_d).sum():
            temp_skeleton |= img_as_ubyte(markers==i)

    ent_skeleton = temp_skeleton.astype(np.uint8)
    ent_skeleton_split = ent_skeleton & ~(ent_mask_d&~ent_mask)
    ent_mask_dd = cv2.dilate(ent_mask_d, kernel_15c)

    # plt.imshow(ent_skeleton&128 | skeleton&64 | (unit_marker|ent_mask)&64 |
    #            ~wall_and_boundary_mask&32 | bgr[:,:,2]&63)

    ret, markers = cv2.connectedComponents(ent_skeleton_split)
    temp_skeleton = np.zeros_like(markers)

    for i in range(1, ret):
    #     print((markers==i).sum())
        if (
            ((markers==i)&(unit_marker|ent_mask)).sum() >= 10 and
            ((markers==i)&(area_markers>0)).sum() >= 10 or
            ((markers==i)&(area_markers==0)).sum() <= 500
        ) or (
            (markers==i).sum() <= 500 and 
            ((markers==i)&ent_mask_dd).sum() >= 10
        ):
            temp_skeleton |= img_as_ubyte(markers==i)

    indoor_skeleton = temp_skeleton.astype(np.uint8)
    outdoor_skeleton = skeleton_split & ~indoor_skeleton

    # plt.imshow(indoor_skeleton&192 | outdoor_skeleton&64 |
    #            ~wall_and_boundary_mask&32 | bgr[:,:,2]&63)


    #####################################
    # Getting indoor area of the unit   #
    #####################################

    indoor1 = get_watershed(~wall_and_boundary_mask, 
        indoor_skeleton&AREA_UNIT | outdoor_skeleton&1)

    indoor1 = cv2.dilate((indoor1==AREA_UNIT).astype(np.uint8)*255, kernel_5c)
    indoor1_d = cv2.dilate(indoor1, kernel_15c, iterations=2)
    # plt.imshow(indoor1&128 | bgr[:,:,2]&62)


    white_color = {'white': {'HSV': ([0,0,245], [255,10,255])}}
    white_mask = color_dict_mask({'HSV': blur['HSV']}, white_color)
    white_mask_e = cv2.erode(white_mask, kernel_5c)
    # plt.imshow(white_mask_e)

    ret, zones = cv2.connectedComponents(~wall_and_boundary_mask)
    # plt.imshow(zones)

    temp_zones = np.zeros_like(zones)


    for i in range(1, ret):
        if ( 
            ((zones==i)&indoor1).sum() == 0 and
            ((zones==i)&(area_markers>0)&indoor1_d).sum() >= 10
        ):
            temp_zones |= img_as_ubyte(zones==i)
    # plt.imshow(temp_zones)


    indoor2 = cv2.dilate(temp_zones.astype(np.uint8), kernel_5c)
    # plt.imshow(indoor2&128 | indoor1&64 | bgr[:,:,2]&63)


    #####################################
    # Getting area of the unit plan     #
    #####################################

    indoor_mask = indoor1|indoor2
    indoor_wall_mask = cv2.dilate(indoor_mask, kernel_15c, iterations=1) & wall_mask
    # plt.imshow(indoor_wall_mask&63 | wall_mask&64)

    _, contours, _ = cv2.findContours( \
            indoor_mask|indoor_wall_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    unit_mask = np.zeros_like(indoor_mask)
    cv2.drawContours(unit_mask, contours, -1, 255, -1)
    # plt.imshow(unit_mask&128 | wall_mask&64)

    
    #####################################
    # Pack the return masks             #
    #####################################
    
    area_markers |= unit_mask & AREA_UNIT
    area_markers |= wall_mask&unit_mask & AREA_WALL
    
    combined_mask = area_markers.astype(np.uint8)
    
    unit_mask_dict = {name: ((area_markers&enum>0)*255).astype(np.uint8)
                      for name, enum in {'unit': AREA_UNIT,
                                         'wall': AREA_WALL,
                                         'entrance': AREA_ENTRANCE,
                                         'LDK': AREA_LDK,
                                         'bedroom': AREA_BEDROOM,
                                         'balcony': AREA_BALCONY,
                                         'bathroom': AREA_BATHROOM,
                                        }.items()}
    
    return unit_mask_dict, combined_mask



# main

In [ ]:
parent_dir = '~/PycharmProjects/NaverApartmentScraper/'

dir_ID_from = 'fp_img'
exp_ID_from = expanduser(parent_dir+dir_ID_from+'/')

dir_hof = 'hall_of_fame'
exp_dir_hof = expanduser(parent_dir+dir_hof+'/')

dirs_ID_exclude = ['fp_img_exclude', 'fp_img_multi-level', 'fp_img_multi-unit']

dir_from = 'fp_img'
exp_dir_from = expanduser(parent_dir+dir_from+'/')

dir_to = 'fp_img_0318'
exp_dir_to = expanduser(parent_dir+dir_to+'/')
ext_to = '.png'

path_fp = 'floorplans.csv'
exp_path_fp = expanduser(parent_dir+path_fp)


IDs = []

# ### adding all of the plans
# IDs += [splitext(f.name)[0] for f in scandir(exp_ID_from) if f.is_file()]

### adding random sample
random.seed('euisoon0318')
from_IDs = [splitext(f.name)[0] for f in scandir(exp_dir_from) if f.is_file()]
IDs += random.sample(from_IDs, 100)

### adding hall of famers
hof_IDs = [splitext(f.name)[0] for f in scandir(exp_dir_hof) if f.is_file()]
IDs += hof_IDs

IDs = list(set(IDs))
print(len(IDs))


IDs_excl = []
for dir_excl in dirs_ID_exclude:
    exp_dir_excl = expanduser(parent_dir+dir_excl+'/')
    IDs_excl += [splitext(f.name)[0] for f in scandir(exp_dir_excl) if f.is_file()]

IDs = list(set(IDs) - set(IDs_excl))
print(len(IDs))


ext_from_dict = {splitext(f.name)[0]: splitext(f.name)[1] \
                 for f in scandir(exp_ID_from) if f.is_file()}


_, fp_list = read_from_csv(exp_path_fp)
bedroom_dict = {i[0]+'_'+i[1]: int(i[4]) for i in fp_list} # {ID: N of bedrooms}

paths_from = {ID: exp_dir_from+ID+ext_from_dict[ID] for ID in IDs}
paths_to = {ID: exp_dir_to+ID+ext_to  for ID in IDs}

makedirs(exp_dir_to, exist_ok=True)



def process_image(path_from, path_to, ext_to='.png'):
    # print(ID)
    
    bgr = read_bgr_from_image_unicode(path_from)
        
    ### get mask
    
    unit_dict, unit_comb = get_unit_mask(bgr)
    
    mask_bgr = np.zeros_like(bgr)
    
    mask_bgr |= unit_comb[:,:,None]
    
    save_bgr_to_image_unicode(mask_bgr, path_to, ext_to)


########################################################################
#    
# [process_image(paths_from[ID], paths_to[ID], ext_to) for ID in IDs]

    
for ID in tqdm_notebook(IDs, desc='Processing plans'):
    process_image(paths_from[ID], paths_to[ID], ext_to)

143
136


HBox(children=(IntProgress(value=0, description='Processing plans', max=136), HTML(value='')))